# Algoritmo de regresión Logística

Antes de iniciar con los modelos de regresion debemos de recordar que en un algoritmo de regresion, nuestra meta es obtener número, (a diferencia de las clasificaciones donde se categorizan los datos). Para esto abra un elemento muy importante que debemos conocer.

## Error cuadrático medio o (MSE)

¿Qué métrica es la más adecuada para una tarea de regresión? ¡El error cuadrático medio!

Para encontrar el EMC, primero debes calcular el error de cada observación:

$$ \text{Error de observación} = \text{Predicción del modelo} - \text{Respuesta correcta} $$

Lo cuál se transforma en la siguiente formula

$$ EMC = \frac{\text {Suma de los cuadrados de los errores de observación}}{\text{Número de observaciones}} $$

Vamos a analizar estos cálculos:

1. El error de observación muestra el grado de discrepancia entre la respuesta correcta y la predicción. Si el error es mucho más grande que cero, el modelo ha sobrevalorado el apartamento; si es mucho menor que cero, entonces el modelo le ha dado un precio reducido.
2. No tendría caso sumar los errores como están, ya que los positivos anularían los negativos. Para hacer que todos cuenten, necesitamos deshacernos de los signos elevándolos al cuadrado.
3. Encontramos la media para obtener datos para todas las observaciones.

Recuerda que al elevar al cuadrado el EMC no tendrá la misma unidad de medida que nuestra variable objetivo por lo que será necesario usar la RECM, que significa raíz del error cuadrático medio. Tiene las mismas unidades de medida que la variable objetivo.

Ahora verás como usarlo en python.

In [ ]:
def mse(answers, predictions):
    result = []
    for i in range(0, len(answers)):
        result.append((answers[i] - predictions[i])**2)
    return sum(result) / len(result)

answers = [623, 253, 150, 237]
predictions = [649, 253, 370, 148]

print(mse(answers, predictions))


Esta fue la forma de utilizar python para poder aplicar la formula, sin embargo python ofrece a través de sklearn una opcion para hacerlo mas facil a traves de la librería `mean_squared_error`.

In [ ]:
from sklearn.metrics import mean_squared_error as mse

answers = [623, 253, 150, 237]
predictions = [649, 253, 370, 148]

mse(answers,predictions)


Ahora veamoslo con una base de datos

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

# Importamos la libreria
df = pd.read_csv('D:/Tripleten/datasets/train_data_us.csv')

# Definimos nuestra base de entrenamiento y nuestra base de resultados
features = df.drop(['last_price'], axis=1)
target = df['last_price'] / 100000

# Calculamos la prediccion promediando el target
predictions = pd.Series(target.mean(), index=target.index)
mse = mean_squared_error(target, predictions)

print('MSE:', mse)
rmse = np.sqrt(mse)
print('RMSE:', rmse)

A continuación veremos su uso real con los diferentes algoritmos de regresión.

## Arbol de decision de regresión

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Importamos nuestra libreria
df = pd.read_csv('D:/Tripleten/datasets/train_data_us.csv')

# Definimos nuestra base de entrenamiento y nuestro resultado esperado (target/test)
train = df.drop(['last_price'], axis=1)
test = df['last_price'] / 100000

# Segmentamos nuestras bases en dos para poder hacer un comparativo y tener datos de entrenamiento y de valicación (test)
x_train, x_test, y_train, y_test = train_test_split(
train, test, test_size= 0.25, random_state=12345) # segmenta el 25% de los datos para hacer el conjunto de validación

# Para poder iterar en un arbol de regresion, primero definimos las siguientes variables
best_model = None
best_result = 10000
best_depth = 0
for depth in range(1, 6): # selecciona el rango del hiperparámetro
    model = DecisionTreeRegressor( max_depth = depth ,random_state=12345 )# entrena el modelo en el conjunto de entrenamiento
    model.fit(x_train, y_train) # entrena el modelo en el conjunto de entrenamiento
    predictions_valid = model.predict(x_test) # obtén las predicciones del modelo en el conjunto de validación
    
    mse = mean_squared_error(y_test,predictions_valid) # calcula la RECM en el conjunto de validación
    recm = np.sqrt(mse)
    if recm < best_result:
        best_model = model
        best_result = recm
        best_depth = depth

print(f"RECM del mejor modelo en el conjunto de validación (max_depth = {best_depth}): {best_result}")

A continuación se presenta el mismo ejercicio pero con otro nombre en las variables con el fin de relacionar las diferentes nominaciones.

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df = pd.read_csv('D:/Tripleten/datasets/train_data_us.csv')

features = df.drop(['last_price'], axis=1)
target = df['last_price'] / 1000000

# En este caso las nombramos features y target en lugar de x_train y y_train
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345) # segmenta el 25% de los datos para hacer el conjunto de validación

best_model = None
best_result = 10000
best_depth = 0
for depth in range(1, 6): # selecciona el rango del hiperparámetro
    model = DecisionTreeRegressor(max_depth=depth, random_state=12345) # inicializa el constructor de modelos con los parámetros random_state=12345 y max_depth=depth
    model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
    predictions_valid = model.predict(features_valid) # obtén las predicciones del modelo en el conjunto de validación
    result = mean_squared_error(target_valid, predictions_valid)**0.5 # calcula la RECM en el conjunto de validación
    if result < best_result:
        best_model = model
        best_result = result
        best_depth = depth

print(f"RECM del mejor modelo en el conjunto de validación (max_depth = {best_depth}): {best_result}")

Ahora procederemos al siguiente algoritmo de regresion.

## Bosque Aleatorio de Regresión

El bosque aleatorio de regresión no difiere mucho del de clasificación. Lo que hace es entrenar a un grupo de árboles independientes y luego promedia sus respuestas para tomar una decisión.

Se trabaja bastante similar al Bosque aleatorio de clasificación, con unas pequeñas diferencias:

- El objetivo (o test) lo definiremos con resultado numérico.
- Usaremos el mse() en lugar del accuracy_score()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Importo la tabla de pandas
df = pd.read_csv('D:/Tripleten/datasets/train_data_us.csv')

train = df.drop('last_price', axis=1)
test = df['last_price'] / 100000

x_train, x_test, y_train, y_test = train_test_split(train,test, test_size=0.25 , random_state=54321)

best_error = 10000
best_est = 0
best_depth = 0

for est in range(10,201,10):
    for depth in range( 1 , 11):
        model = RandomForestRegressor(n_estimators= est, max_depth = depth, random_state=54321)
        model.fit(x_train, y_train)
        predictions_valid = model.predict(x_test)
        mse = mean_squared_error(y_test, predictions_valid)
        rmse = np.sqrt(mse)
        print("Validación rmse para los n_estimators de", est, ", depth=", depth, "is", rmse)
        if rmse < best_error:
            best_error = rmse
            best_est = est
            best_depth = depth

print("RMSE del mejor modelo en el conjunto de validación:", best_error, "n_estimators:", best_est, "best_depth:", best_depth)


Volvamos a resolver el ejemplo anterior pero variando los nombres de las variables para entender mejor su significado

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

df = pd.read_csv('D:/Tripleten/datasets/train_data_us.csv')

df_train, df_valid = train_test_split(df, test_size=0.25, random_state=54321)

features_train = df_train.drop(['last_price'], axis=1)
target_train = df_train['last_price'] / 100000
features_valid = df_valid.drop(['last_price'], axis=1)
target_valid = df_valid['last_price'] / 100000

best_error = 10000
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestRegressor(random_state=54321, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        error = mean_squared_error(target_valid, predictions_valid)**0.5
        # print("Validación de RECM para los n_estimators de", est, ", depth=", depth, "es", error)
        if error < best_error:
            best_error = error
            best_est = est
            best_depth = depth

print("RECM del mejor modelo en el conjunto de validación:", best_error, "n_estimators:", best_est, "best_depth:", best_depth)


## Regresion Lineal

La idea general de la regresión lineal es que hay una sencilla ecuación lineal que describe cómo un determinado parámetro define el output. Es un método utilizado para modelar la relación entre una variable dependiente y una o más variables independientes al aproximar esta relación con una función lineal.

En resumen, LinearRegression de sklearn proporciona una forma conveniente de ajustar modelos de regresión lineal y realizar predicciones basadas en esos modelos en Python.

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

df = pd.read_csv('D:/Tripleten/datasets/train_data_us.csv')

features = df.drop('last_price', axis=1)
target = df['last_price']/100000

features_train, features_valid, target_train, target_valid = train_test_split(
    features,target, test_size=0.25 , random_state=12345) # segmenta el 25% de los datos para hacer el conjunto de validación

model = LinearRegression()
model.fit(features_train,target_train) # entrena el modelo en el conjunto de entrenamiento
predictions_valid = model.predict(features_valid) # obtén las predicciones del modelo en el conjunto de validación
mse = mean_squared_error(target_valid, predictions_valid)
result = np.sqrt(mse)
print("RECM del modelo de regresión lineal en el conjunto de validación:", result)


In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor # importa el modelo que escogiste desde este módulo

# df = pd.read_csv('/datasets/train_data_us.csv')


# inicializa las variables
x_train = df.drop(columns=['last_price']) 
x_test = df['last_price']


final_model = DecisionTreeRegressor( max_depth=3 , random_state =54321 )# inicializa el constructor para el modelo que tenía el mejor valor de RECM
final_model.fit(x_train,x_test) # entrena el modelo en el conjunto de entrenamiento



In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

df = pd.read_csv('D:/Tripleten/datasets/train_data_us.csv')

# inicializa las variables
x_train = df.drop(columns=['last_price']) 
x_test = df['last_price']/100000

final_model = DecisionTreeRegressor( max_depth=10 , random_state =54321 )# inicializa el constructor para el modelo que tenía el mejor valor de RECM

final_model.fit(x_train,x_test) # entrena el modelo en el conjunto de entrenamiento
predictions = final_model.predict(x_train)
mse = mean_squared_error(x_test, predictions)
rmse = np.sqrt(mse)

print(rmse)

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

df = pd.read_csv('D:/Tripleten/datasets/train_data_us.csv')

# inicializa las variables
x_train = df.drop(columns=['last_price']) 
x_test = df['last_price']/100000

final_model = RandomForestRegressor(n_estimators=100 ,max_depth=10 , random_state =54321 )# inicializa el constructor para el modelo que tenía el mejor valor de RECM

best_est = 0
best_depth = 0
best_rmse = 10000

for est in range(1 , 100):
    for depth in range(1 ,10 ):
        model.fit(x_train,x_test)
        predictions = model.predict(x_train)
        mse = mean_squared_error(x_test ,predictions)
        rmse = mse**0.5
        print("Actual:", rmse, "n_estimators:", est, "best_depth:", depth)

        if rmse < best_rmse:
            best_est = est
            best_depth = depth
            best_rmse = rmse

print("RECM del mejor modelo en el conjunto de validación:", best_rmse, "n_estimators:", best_est, "best_depth:", best_depth)

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

df = pd.read_csv('D:/Tripleten/datasets/train_data_us.csv')

# inicializa las variables
x_train = df.drop(columns=['last_price']) 
x_test = df['last_price']/100000

final_model = RandomForestRegressor(n_estimators=40 , random_state = 54321 )# inicializa el constructor para el modelo que tenía el mejor valor de RECM
model.fit(x_train,x_test)
predictions = model.predict(x_train)
mse = mean_squared_error(x_test ,predictions)
rmse = mse**0.5

print("RECM del mejor modelo en el conjunto de validación:", best_rmse, "n_estimators:", best_est, "best_depth:", best_depth)

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv('D:/Tripleten/datasets/train_data_us.csv')

features = df.drop(['last_price'], axis=1)
target = df['last_price'] / 100000

final_model = RandomForestRegressor(random_state=54321, n_estimators=40)
final_model.fit(features, target)

predictions = final_model.predict(features)
result = mean_squared_error(target, predictions)**0.5 
print(result)